In [ ]:
require(cowplot)
require(data.table)
require(ggplot2)
require(pbapply)
pboptions(type="timer")

### Read input region metadata.

In [ ]:
probe_meta = fread("data/rois/all_regions.tsv")
probe_summary = fread("data/probe_summary_probelet.tsv")    # data/probe_summary.csv
probelist = list.files("data/", recursive=TRUE, pattern="probe_.+oligos\\.tsv$")

current_region = 1

current_probe_id = 3    # 5 <-> 1e-5 etc
probe = fread(paste("data",probelist[current_probe_id],sep="/"))

probelist[current_probe_id]
head(probe)

In [ ]:
probe_meta[current_region]
#probe_summary[, region_start := probe_meta[current_region, DNA_start], region_end := probe_meta[current_region, DNA_end]]

### Evaluate oligo distribution homogeneity

In [ ]:

probe[, homogeneity := 1 / sd(start[2:.N] - end[1:(.N-1)])]
probe[, summary(homogeneity)]
probe[, Tm_range := diff(range(Tm))]
probe[, summary(Tm_range)]

### Visualize current probe and feature stats

In [ ]:
options(repr.plot.width=20, repr.plot.height=2)
ggplot(probe, aes(x=start/1e6, xend=end/1e6, y=1, yend=1)) + geom_segment() +
    theme_bw() + labs(x="Genomic coordinates (Mb)") + theme(axis.text.y=element_blank(), axis.ticks.y=element_blank(), axis.title.y=element_blank()) +
    geom_segment(aes(x=(start+end)/2e6, xend=(start+end)/2e6, y=.9, yend=1.1), linetype="dashed", size=.25)

In [ ]:
print("Stats for Tm")
print(sprintf("Tm range: %f", probe[, Tm_range[1]]))
probe[, summary(Tm)]
options(repr.plot.width=4, repr.plot.height=4)
ggplot(probe, aes(x=Tm)) + geom_histogram(bins=30, color="#323232") + labs(x="Melting temperature (degC)") + theme_bw()

In [ ]:
print("Stats for distance between consecutive oligos")
probe[, summary(start[2:.N]-end[1:(.N-1)])]
options(repr.plot.width=4, repr.plot.height=4)
ggplot(probe[, .(odist=start[2:.N]-end[1:(.N-1)])]) + geom_histogram(aes(x=odist), color="#323232", bins=30) +
    labs(x="Distance between consecutive oligos (bp)", subtitle=sprintf("Mean dist: %f bp", probe[, mean(start[2:.N]-end[1:(.N-1)])])) + theme_bw()

In [ ]:
print("Stats for off_targets")
probe[, summary(offtargets_no)]
probe[, .N, by=offtargets_no][order(offtargets_no)]